In [ ]:
from serpapi import GoogleSearch
from dataExtraction import extract
from IPython.display import clear_output
from pprint import pprint

import pandas as pd
from transformers import pipeline
import re
import statistics
import os

## Data Collection

In [ ]:
#########################################################
# WARNING!                                              #
# Don't run with this API key unless 100% required      #
#########################################################

# API Key Store
# Travis: 7b49b198668e629ad2ea004850f238d18e56e6e6f3ed19a4d71430a670f4ba29
# Leon: a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada
# Konni: f69dfcb654a45b2c1da7b024ebec56337ea4634e63d63765caff2397d2594c12
# Nicole: ebd5e2d7f66e44eaa6b4cb32c0c0ffa8d82cbc49f1b6dcd286573fcafcf5282d

In [ ]:
# Check all of the below 

api_key = "a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada"
country = "de" # https://serpapi.com/google-countries
location = "Germany" # Word form of the above

start_page = 0                     # If you have conducted one search, but want to get more, set to last result 
number_of_searches = 40
end_page = number_of_searches * 10

In [ ]:
for i in range(start_page, end_page, 10):
    params = {
        "engine": "google_jobs",
        "q": "data scientist",
        "hl": "en",
        "api_key": api_key,
        "gl": country,
        "location": location,
        "start": i 
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    
    try:
        extract(results)
    except KeyError:
        pprint(results)
        break
    
    clear_output(wait=True)
    percent = int((i/end_page) * 100)
    print(percent*'#', f" {percent}%")
    
print(f"{number_of_searches} searches completed\n" +
f"country: {country}\n" +
f"start page: {start_page}\n" +
f"end page: {end_page}")

## Salary Extraction

In [9]:
df = pd.read_csv('data.csv')
df.head(10)

,title,company_name,location,via,description,job_highlights,detected_extensions,job_id
0,Junior Data Scientist,ING,Amsterdam,ING Careers,As the data driven mindset is more and more em...,['As the data driven mindset is more and more ...,"{'posted_at': '6 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBTY2llbnRpc3...
1,"JUNIOR DATA SCIENTIST - Dubai, UAE",Cobblestone Energy,Utrecht,LinkedIn,"Location: Dubai, UAE (We provide visa sponsors...","[""Location: Dubai, UAE (We provide visa sponso...","{'posted_at': '4 hours ago', 'schedule_type': ...",eyJqb2JfdGl0bGUiOiJKVU5JT1IgREFUQSBTQ0lFTlRJU1...
2,Data Scientist Mobiliteit,TNO,The Hague,TNO,Halen we in Nederland de klimaatdoelen op het ...,['Halen we in Nederland de klimaatdoelen op he...,"{'posted_at': '5 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBNb2JpbG...
3,Data Scientist Real Estate for Catella Investm...,Catella Investment Management Benelux,Maastricht,Limburgvac,As a Data Scientist in the Research & Investme...,['As a Data Scientist in the Research & Invest...,"{'posted_at': '20 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBSZWFsIE...
4,Data Scientist,Effectory,Amsterdam,Effectory Jobs,Improving the working lives of millions of peo...,['Improving the working lives of millions of p...,{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...
5,Data Scientist,Adyen,Amsterdam,Nationale Vacaturebank,Functieomschrijving Data Analytics Amsterdam T...,"[""Functieomschrijving\n\nData Analytics Amster...","{'posted_at': '17 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...
6,Data Scientist bij Transavia,Transavia,Schiphol,Vacatures - Transa,Wij zoeken jou als Data Scientist Voor ons Str...,['Wij zoeken jou als Data Scientist\n\nVoor on...,{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBiaWogVH...
7,Data Science Lead - Amsterdam,Bynder,Amsterdam,Careers At Bynder,Bynder goes far beyond managing digital assets...,['Bynder goes far beyond managing digital asse...,"{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgTGVhZCAtIE...
8,"LEAD DATA SCIENTIST - Dubai, UAE",Cobblestone Energy,Rotterdam,LinkedIn,Employment type: Full-time & Permanent Reports...,"[""Employment type: Full-time & Permanent\n\nRe...","{'posted_at': '1 day ago', 'schedule_type': 'F...",eyJqb2JfdGl0bGUiOiJMRUFEIERBVEEgU0NJRU5USVNUIC...
9,Data Science and Artificial Intelligence Fello...,Wageningen University & Research,Wageningen,AcademicTransfer,Are you a computer scientist with a PhD degree...,['Are you a computer scientist with a PhD degr...,{'posted_at': '2 days ago'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgYW5kIEFydG...


In [ ]:
def extract_salary():
    df['job_highlights'] = df['job_highlights'].replace(r'\n\n|\n•|\n|\\n|\\n•|•', '', regex=True)  # Remove new line char
    df['job_highlights'] = df['job_highlights'].str.slice(2, -2)  # Remove [] and additional "" marks
    
    qa_model = pipeline("question-answering", 
                    model='distilbert-base-cased-distilled-squad') # Define the model 
    question = "What is the salary or salary range for the job?"  # Define the quesiton to be answered
    df['salary'] = df.apply(lambda x: "Not available" if qa_model(question=question, 
                                                                           context=(x['job_highlights'] + 
                                                                                    x['title'] + 
                                                                                    x['description']))['score'] < 0.3
                                                                else qa_model(question=question, 
                                                                              context=(x['job_highlights'] + 
                                                                                       x['title'] + 
                                                                                       x['description']))['answer'], axis=1)
    return df
    
df = extract_salary()

In [ ]:
def find_specified_salary(x):
    df['salary'] = df['salary'].str.replace(',', '', regex=True)  # Replace comma point in numerical values
    df['salary'] = df['salary'].str.replace('.', '', regex=True)  # Replace dot point in numerical values
    
    values = re.findall(r'\d+', str(x))  # Identify numerical values
    salary = 'Not available'
    salary_range = 'Not available'
    if len(values) == 1:  # Single numerical value, not a range
        salary = int(values[0]) 
        
    elif len(values) == 2:  # Two numerical values indicate a range
        min_salary = int(values[0])  # Min of salary range
        max_salary = int(values[1])  # Max of salary range
        salary = statistics.median([min_salary, max_salary])  # Calculate range median
        salary_range = str(min_salary) + "-" + str(max_salary)  # Format range
        
    return pd.Series([salary, salary_range])


df[['salary', 'salary_range']] = df['salary'].apply(find_specified_salary)

In [ ]:
currency_mapping = {'AUD': 'AUD', 
                    'EUR': 'EUR', 
                    'JPY': 'JPY', 
                    'CHF': 'CHF', 
                    'USD': 'USD',
                    'GBP': 'GBP',
                    'dollar': 'USD', 
                    'euro': 'EUR', 
                    'pound': 'GBP', 
                    'default': 'EUR'}

currency_indicators = ['AUD', 'GBP', 'EUR', 'JPY', 'CHF', 'USD', 'dollar', 'euro', 'pound'] # 6 dominant country codes    


def identify_currencies():
    regex_pattern = '|'.join([f'{key_word}' for key_word in currency_indicators])  # Create regex pattern
    search_space = df['description'] + df['title'] + df['job_highlights']  # Concatenate key strings into a single search space
    currencies = search_space.map(lambda x:currency_mapping['default'] if len(re.findall(regex_pattern, x)) == 0
                             else re.findall(regex_pattern, x)[0])  # Perform regex search in search space
    df['currency'] = currencies  # Assign determined currency
    return df
    
    
df = identify_currencies()

In [ ]:
df.to_csv('data_salaries.csv')

## Skill Extraction

In [ ]:
# Add empty columns
skills_list = []
with open("notes_and_prototypes/skills_list.txt") as file:
    while (line := file.readline().rstrip()):
        if line == "R":
            skills_list.append(" R ")
        else:
            skills_list.append(line)


# Replace blank spaces with underscores for dataframe columns
columns_list = []
for element in skills_list:
    columns_list.append(element.replace(" ", "_"))

for element in columns_list:
    df[element] = 0

print(skills_list)

df.head(5)

In [ ]:
# Find keywords and set columns accordingly

for i, row in df.iterrows():
    for keyword in skills_list:
        if keyword.lower() in df.at[i, "description"].lower():
            col = keyword.replace(" ", "_")
            df.at[i, col] = 1
# Print first results to check
df.head(10)

In [ ]:
# Export to csv
df.to_csv('data_skills.csv')